# Dependencies

In [1]:
using CSV, DataFrames, Random

In [2]:
using Flux

In [3]:
using CUDA

In [4]:
using Plots

In [5]:
plotlyjs()
theme(:wong);

WebIO._IJuliaInit()

# Data loading

In [6]:
function splitdf(df, pct)
   @assert 0 <= pct <= 1
   ids = collect(axes(df, 1))
   shuffle!(ids)
   sel = ids .<= nrow(df) .* pct
   return view(df, sel, :), view(df, .!sel, :)
end;

In [7]:
df = "fashion-mnist_train.csv" |> CSV.File |> DataFrame;
df_train, df_valid = splitdf(df, 0.666)
df_test = "fashion-mnist_test.csv" |> CSV.File |> DataFrame;

In [8]:
labels = Dict(
    0 => "T-shirt",
    1 => "Trouser",
    2 => "Pullover",
    3 => "Dress",
    4 => "Coat",
    5 => "Sandal",
    6 => "Shirt",
    7 => "Sneaker",
    8 => "Bag",
    9 => "Ankle boot"
);

In [9]:
function load_y(df)
    y = df[:, :label]
    Flux.onehotbatch(y, 0:9)
end

y_train = load_y(df_train) |> y -> Flux.label_smoothing(y, 0.2f0) |> gpu
y_valid = load_y(df_valid) |> gpu;
y_test= load_y(df_test) |> gpu;

In [10]:
function load_x(df)
    x = select(df, Not(:label)) |> Matrix |> permutedims
    x = reshape(x, 28, 28, 1, :) / 255
    convert(Array{Float32,4}, x) |> m -> permutedims(m, (2,1,3,4))
end

x_train = load_x(df_train) |> gpu
x_valid = load_x(df_valid) |> gpu;
x_test = load_x(df_test) |> gpu;

# Custom training function

In [11]:
function train!(loss, ps, data, opt)
	local training_loss
	for d in data
    	gs = gradient(ps) do
      		training_loss = loss(d...)
      		return training_loss
		end
    # Insert whatever code you want here that needs training_loss, e.g. logging.
    # E.g. logging with TensorBoardLogger.jl as histogram so you can see if it is becoming huge.
    Flux.update!(opt, ps, gs)
    # Here you might like to check validation set accuracy, and break out to do early stopping.
	end
    training_loss
end;

In [12]:
predict(x) = labels[Flux.onecold(model(cat(x; dims=4)), 0:9)[1]]
get_label(y) = labels[Flux.onecold(y, 0:9)[1]]

function get_acc(model)
    function acc(x, y) 
        ŷ = model(x) |> Flux.onecold
        cy = y |> Flux.onecold
        sum(ŷ .== cy) / length(cy)
    end
end;

# CNN

In [13]:
model = Chain(
    Conv((3, 3), 1 => 32, relu)
    , AdaptiveMaxPool((14, 14))
    , Conv((3, 3), 32 => 32, relu)
    , Flux.Dropout(0.2)
    , Flux.flatten
    , Dense(4608, 32, relu)
    , Dense(32, 10)
    , softmax
) |> gpu

ps = params(model)

loss(x, y) = Flux.mse(model(x), y)

opt = ADAM()

data = Flux.DataLoader((x_train, y_train), batchsize=512, shuffle=true) |> gpu;

In [14]:
tre = []
# vde = []
# tse = []
# ats = []
cnn_acc = get_acc(model)
@time for epoch=1:75
	push!(tre, train!(loss, ps, data, opt))
	# push!(vde, loss(x_valid, y_valid))
	# push!(tse, loss(x_test, y_test))
	# push!(ats, cnn_acc(x_test, y_test))
end
plot(
    plot(tre, title="Training error")
    # , plot(vde, title="Validation error")
    # , plot(tse, title="Test error")
    # , plot(ats, title="Test accuracy")
    , legend = false
)

280.734168 seconds (222.27 M allocations: 12.496 GiB, 1.91% gc time, 27.30% compilation time)


## Evaluation

In [15]:
cnn_acc(x_test, y_test)

0.8941

In [33]:
idx = 21
xi = x_test[:, :, 1, idx]
ŷi = xi |> predict
yi = y_test[:, idx] |> get_label
plot(Gray.(xi), title="y=$(yi); ŷ=$ŷi")

# Autoencoder

In [17]:
x_train_autoenc = select(df_train, Not(:label)) |> Matrix |> permutedims |> m -> convert(Matrix{Float32}, m / 255) |> gpu;

In [18]:
x_test_autoenc = select(df_test, Not(:label)) |> Matrix |> permutedims |> m -> convert(Matrix{Float32}, m / 255) |> gpu;

## Compression and decompression

In [19]:
enc = Chain(
    Dense(784, 1024, relu)
    , Dense(1024, 256, relu)
    , Dense(256, 16, relu)
) |> gpu

knw = Chain(
    Dense(16, 16, σ)
) |> gpu

dec = Chain(
    Dense(16, 256, relu)
    , Dense(256, 1024, relu)
    , Dense(1024, 784, relu)
) |> gpu

auto = Chain(
    enc
    , knw
    , dec
) |> gpu;

## Classification

In [20]:
clf = Chain(
    Dense(16, 32, relu)
    , Dense(32, 10)
    , softmax
) |> gpu

m2 = Chain(
    enc
    , knw
    , clf
) |> gpu;

In [21]:
data_autoenc = Flux.DataLoader((x_train_autoenc, x_train_autoenc), batchsize=1024, shuffle=true) |> gpu;

In [22]:
loss_autoenc(x, y) = Flux.mse(auto(x), y)

loss_autoenc (generic function with 1 method)

In [23]:
h = []
@time for e=1:200
    push!(h, train!(loss_autoenc, params(auto), data_autoenc, ADAM()))
end;

 84.918757 seconds (46.16 M allocations: 3.034 GiB, 2.15% gc time, 3.00% compilation time)


In [24]:
h |> plot

In [25]:
og = x_test_autoenc[:, 97]
en = enc(og)
kn = knw(en)
dc = dec(kn)
plot(
    plot(
        Gray.(reshape(og, 28, 28)'), title="Original"
    ),
    plot(
        Gray.(reshape(en, 4, 4)'), title="Encoded"
    ),
    plot(
        Gray.(reshape(kn, 4, 4)'), title="Knowledge"
    ),
    plot(
        Gray.(reshape(dc, 28, 28)'), title="Decoded"
    )
)

In [26]:
data_autoenc_classif = Flux.DataLoader((knw(enc(x_train_autoenc)), y_train), batchsize=1024, shuffle=true) |> gpu;

In [27]:
loss_autoenc_classif(x, y) = Flux.mse(clf(x), y)

loss_autoenc_classif (generic function with 1 method)

In [28]:
hc = []
@time for e=1:200
    push!(hc, train!(loss_autoenc_classif, params(clf), data_autoenc_classif, ADAM()))
end;

  5.940014 seconds (13.44 M allocations: 913.042 MiB, 2.73% gc time, 6.07% compilation time)


In [29]:
hc |> plot

## Evaluation

In [30]:
compressed_x = x_test_autoenc |> enc |> knw

16×10000 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0651229  0.274439   0.499075  0.418661   …  0.234892   0.232131   0.112543
 0.465797   0.12246    0.388423  0.398511      0.783565   0.659216   0.709925
 0.287623   0.117335   0.414861  0.600785      0.367044   0.0940424  0.272906
 0.741685   0.472775   0.453884  0.691717      0.736088   0.672745   0.708024
 0.8531     0.151008   0.290375  0.76282       0.969064   0.65075    0.324181
 0.374478   0.544602   0.493669  0.267957   …  0.0472209  0.470565   0.185393
 0.323961   0.0483421  0.432303  0.809526      0.323545   0.569212   0.397032
 0.596084   0.622977   0.581721  0.63131       0.0196864  0.67345    0.493463
 0.059447   0.374696   0.428579  0.534111      0.791346   0.383902   0.386726
 0.511642   0.379977   0.521735  0.117279      0.0466268  0.226306   0.267165
 0.141803   0.155511   0.35221   0.0412939  …  0.683873   0.281963   0.185318
 0.820063   0.256076   0.615308  0.652788      0.599505   0.677847   0.407158
 0.177142  

In [31]:
autoenc_preds = compressed_x |> clf |> Flux.onecold;

In [32]:
accuracy_autoenc = get_acc(clf)
accuracy_autoenc(compressed_x, y_test)

0.8392